In [1]:
import os
os.chdir('test_split')
import geojson
from dateutil.parser import parse
from glob import glob

In [12]:
def split_geojson(input_geojson, split_minutes=5):
    with open(input_geojson, 'r') as f:
        input_points = geojson.load(f)
        
    basename = os.path.basename(input_geojson).replace(".geojson", "")
        
    write_first_point(input_points, basename, split_minutes)
    write_lines(input_points, basename, split_minutes)
    write_interval(input_points, basename, interval=10, split_minutes=split_minutes, name='10min')
    write_interval(input_points, basename, interval=60, split_minutes=split_minutes, name='Hourly')

In [21]:
def write_first_point(input_points, basename, split_minutes):
    first_point = geojson.FeatureCollection([input_points[0]])
    time = parse(input_points[0].properties['time'])
    time_str = f"{time.hour:02}{(time.minute // split_minutes) * split_minutes:02}"
    output_filename = basename + f"_FirstPoint_{time_str}.geojson"
    with open(output_filename, 'w') as f:
        geojson.dump(first_point, f)

In [22]:
def feature_from_line_coords(segment):
    line_string = geojson.LineString(coordinates=segment)
    feature = geojson.Feature(geometry=line_string)
    
    return feature

In [23]:
def write_lines(input_points, basename, split_minutes):
    line_segments = {}
    current_line_seg = []

    # Split the lines
    for feature in input_points.features:
        coords = feature.geometry.coordinates
        time = parse(feature.properties['time'])

        current_line_seg.append(coords)
        if time.minute % split_minutes == 0:
            if len(current_line_seg) > 1:
                # If we've got something from before, then add to list of segs
                time_str = f"{time.hour:02}{(time.minute // split_minutes) * split_minutes:02}"
                line_segments[time_str] = current_line_seg
            # Start new line segment
            current_line_seg = [coords]

    # Deal with any left over at the end
    time_str = f"{time.hour:02}{((time.minute // split_minutes) + 1) * split_minutes:02}"
    line_segments[time_str] = current_line_seg

    # Write them out to separate files
    for time, segment in line_segments.items():
        feature = feature_from_line_coords(segment)
        feature_collection = geojson.FeatureCollection([feature])
        with open(basename + '_Lines_' + str(time) + '.geojson', 'w') as f:
            geojson.dump(feature_collection, f)
    

In [29]:
def write_interval(input_points, basename, interval, split_minutes, name):
    points = {}
    current_points = []

    for feature in input_points.features:
        time = parse(feature.properties['time'])

        if time.minute % interval == 0:
            properties = feature.properties
            time = parse(properties['time'])
            properties['time_str'] = f"{time.hour:02}:{time.minute:02}"
            current_points.append(geojson.Feature(geometry=feature.geometry, properties=properties))
            
        if time.minute % split_minutes == 0:
            if len(current_points) > 0:
                time_str = f"{time.hour:02}{(time.minute // split_minutes) * split_minutes:02}"
                points[time_str] = current_points
            current_points = []


    for time, feature in points.items():
        feature_collection = geojson.FeatureCollection(feature)
        with open(basename + '_' + name + "_" + str(time) + '.geojson', 'w') as f:
            geojson.dump(feature_collection, f)

In [30]:
def generate_layer_list(time_strs):
    layer_list = ""
    
    layer_template = """<Layer id="1995-12-12 {time}" name="1995-12-12 {time}">
    <Layer id="Nelson 1995-12-12 {time}" name="Nelson"/>
    <Layer id="Collingwood 1995-12-12 {time}" name="Collingwood"/>
</Layer>"""
    
    
    for time_str in time_strs:
        layer_list += "\n" + layer_template.format(time=time_str)
        
    return layer_list

In [31]:
def generate_content_layer(time_str):
    time_no_colon = time_str.replace(":", "")
    content = ""
    
    content += f'<IfLayerOn layerId="1995-12-12 {time_str}">\n'
    
    content += generate_ship_content_layer(time_str, "Nelson") + '\n\n'
    
    content += generate_ship_content_layer(time_str, "Collingwood")
    
    content += "</IfLayerOn>"
    
    return content

In [40]:
def generate_ship_content_layer(time_str, ship_name):
    time_no_colon = time_str.replace(":", "")
    
    color = SHIP_NAME_TO_COLOR[ship_name]
    content = ""
    
    
    content += '    <IfLayerOn layerId="{ship_name} 1995-12-12 {time}">\n'
    
    if os.path.exists(f"{ship_name}_Lines_{time_no_colon}.geojson"):
        content += """        <Vector dataset="{ship_name}_Lines_{time_no_colon}.geojson" layer="{ship_name}_Lines_{time_no_colon}" georeferencingId="georeferenced" ogrStyleString='PEN(c:{color},w:5px)'/>\n"""
    
    if os.path.exists(f"{ship_name}_10min_{time_no_colon}.geojson"):
        content += """        <Vector dataset="{ship_name}_10min_{time_no_colon}.geojson" layer="{ship_name}_10min_{time_no_colon}" georeferencingId="georeferenced" ogrStyleString='SYMBOL(c:{color},s:2,id:"ogr-sym-3")'>
                <LogicalStructure displayLayerName="{ship_name}" fieldToDisplay="time"/>
        </Vector>\n"""
        
    if os.path.exists(f"{ship_name}_Hourly_{time_no_colon}.geojson"):
        content += """        <VectorLabel dataset="{ship_name}_Hourly_{time_no_colon}.geojson" layer="{ship_name}_Hourly_{time_no_colon}" georeferencingId="georeferenced" ogrStyleString='LABEL(t:{{time_str}},c:{color},s:24pt,p:4,dx:7mm,bo:1)'/>\n"""
    
    if os.path.exists(f"{ship_name}_FirstPoint_{time_no_colon}.geojson"):
        content += """        <VectorLabel dataset="{ship_name}_FirstPoint_{time_no_colon}.geojson" layer="{ship_name}_FirstPoint_{time_no_colon}" georeferencingId="georeferenced" ogrStyleString='LABEL(t:"{ship_name}",c:{color},s:24pt,p:2,dy:10mm,bo:1)'/>\n"""
    
    content += "    </IfLayerOn>"
    
    return content.format(time=time_str, time_no_colon=time_no_colon, ship_name=ship_name, color=color)

In [35]:
split_geojson("Nelson.geojson", split_minutes=5)

In [36]:
split_geojson("Collingwood.geojson", split_minutes=5)

In [41]:
SHIP_NAME_TO_COLOR = {'Nelson': '#bd1b44',
                      'Collingwood': '#0d0d8e'}

In [42]:
files = glob("*_*_*.geojson")
times_set = {os.path.splitext(filename)[0].split("_")[2] for filename in files}
time_strs = [f"{time[:2]}:{time[2:]}" for time in sorted(times_set)]

layer_list = generate_layer_list(time_strs)

content_layer_strs = [generate_content_layer(time_str) for time_str in time_strs]
whole_content_layer = "\n\n".join(content_layer_strs)

In [43]:
print(layer_list)


<Layer id="1995-12-12 05:00" name="1995-12-12 05:00">
    <Layer id="Nelson 1995-12-12 05:00" name="Nelson"/>
    <Layer id="Collingwood 1995-12-12 05:00" name="Collingwood"/>
</Layer>
<Layer id="1995-12-12 05:05" name="1995-12-12 05:05">
    <Layer id="Nelson 1995-12-12 05:05" name="Nelson"/>
    <Layer id="Collingwood 1995-12-12 05:05" name="Collingwood"/>
</Layer>
<Layer id="1995-12-12 05:10" name="1995-12-12 05:10">
    <Layer id="Nelson 1995-12-12 05:10" name="Nelson"/>
    <Layer id="Collingwood 1995-12-12 05:10" name="Collingwood"/>
</Layer>
<Layer id="1995-12-12 05:15" name="1995-12-12 05:15">
    <Layer id="Nelson 1995-12-12 05:15" name="Nelson"/>
    <Layer id="Collingwood 1995-12-12 05:15" name="Collingwood"/>
</Layer>
<Layer id="1995-12-12 05:20" name="1995-12-12 05:20">
    <Layer id="Nelson 1995-12-12 05:20" name="Nelson"/>
    <Layer id="Collingwood 1995-12-12 05:20" name="Collingwood"/>
</Layer>
<Layer id="1995-12-12 05:25" name="1995-12-12 05:25">
    <Layer id="Nelso

In [44]:
print(whole_content_layer)

<IfLayerOn layerId="1995-12-12 05:00">
    <IfLayerOn layerId="Nelson 1995-12-12 05:00">
        <Vector dataset="Nelson_10min_0500.geojson" layer="Nelson_10min_0500" georeferencingId="georeferenced" ogrStyleString='SYMBOL(c:#bd1b44,s:2,id:"ogr-sym-3")'>
                <LogicalStructure displayLayerName="Nelson" fieldToDisplay="time"/>
        </Vector>
        <VectorLabel dataset="Nelson_Hourly_0500.geojson" layer="Nelson_Hourly_0500" georeferencingId="georeferenced" ogrStyleString='LABEL(t:{time_str},c:#bd1b44,s:24pt,p:4,dx:7mm,bo:1)'/>
        <VectorLabel dataset="Nelson_FirstPoint_0500.geojson" layer="Nelson_FirstPoint_0500" georeferencingId="georeferenced" ogrStyleString='LABEL(t:"Nelson",c:#bd1b44,s:24pt,p:2,dy:10mm,bo:1)'/>
    </IfLayerOn>

    <IfLayerOn layerId="Collingwood 1995-12-12 05:00">
        <VectorLabel dataset="Collingwood_FirstPoint_0500.geojson" layer="Collingwood_FirstPoint_0500" georeferencingId="georeferenced" ogrStyleString='LABEL(t:"Collingwood",c:#0d0d8e